# Electricity Demand and Windpower Data Preparation
The Dataset was downloaded from https://www.gridwatch.templar.co.uk/download.php

We downloaded only demand and wind datasets.

In [1]:
csv_path = '../input/electricity-demand-and-windpower/gridwatch_Demand_wind.csv'

import pandas as pd
import numpy as np
df = pd.read_csv(csv_path, index_col=0)
df.head(5)

,timestamp,demand,wind
id,,,
4,2011-05-27 15:50:04,38874,1253
5,2011-05-27 15:55:02,38845,1304
6,2011-05-27 16:00:02,38745,1322
7,2011-05-27 16:05:01,38826,1364
8,2011-05-27 16:10:02,38865,1370


In [2]:
print(df.shape)

(1174923, 3)


In [3]:
iter_data = 0
history_demand=[]
history_wind =[]
Demand_train = []
Demand_val = []
Demand_test = []
Wind_train = []
Wind_val = []
Wind_test = []
iter_split = 0


for row in df.iloc:
    iter_data = iter_data + 1
    
    if iter_data<5:
        history_demand.append(row[1])
        history_wind.append(row[2])
        continue
        
    time_in_hour = float(row[0][12:14])+float(row[0][15:17])/60+float(row[0][18:20])/3600 
    Approx_day_in_year = float(row[0][9:11]) + float(row[0][6:8])*30
    
    Demand_data = np.zeros(7)
    Demand_data[0:4] = history_demand
    Demand_data[4] = time_in_hour
    Demand_data[5] = Approx_day_in_year
    Demand_data[6] = row[1]
      

    Wind_data = np.zeros(7)
    Wind_data[0:4] = history_wind
    Wind_data[4] = time_in_hour
    Wind_data[5] = Approx_day_in_year
    Wind_data[6] = row[2]
    
    if (np.sum(Demand_data)+np.sum(Wind_data)) <1e20: # Discard Nan or Null values
        if iter_split%10>7:
            Demand_train.append(Demand_data)
            Wind_train.append(Wind_data)
        if iter_split%10<2:
            Demand_val.append(Demand_data)
            Wind_val.append(Wind_data)
        if iter_split%10<8 and iter_split%10>1:
            Demand_test.append(Demand_data)
            Wind_test.append(Wind_data)

        history_demand[0:3] = history_demand[1:4]
        history_demand[3] = row[1]

        history_wind[0:3] = history_wind[1:4]
        history_wind[3] = row[2]
        iter_split = iter_split +1
        if iter_split == 2e4:
            break


In [4]:
df_demand = pd.DataFrame(Demand_train, 
             columns=['y(t-4)','y(t-3)','y(t-2)','y(t-1)','Hour','Day', 'y(t)'])
df_demand.to_csv('./Demand_train.csv',index=False)

df_demand = pd.DataFrame(Demand_val, 
             columns=['y(t-4)','y(t-3)','y(t-2)','y(t-1)','Hour','Day', 'y(t)'])
df_demand.to_csv('./Demand_val.csv',index=False)

df_demand = pd.DataFrame(Demand_test, 
             columns=['y(t-4)','y(t-3)','y(t-2)','y(t-1)','Hour','Day', 'y(t)'])
df_demand.to_csv('./Demand_test.csv',index=False)


df_demand.head()

,y(t-4),y(t-3),y(t-2),y(t-1),Hour,Day,y(t)
0,38745.0,38826.0,38865.0,38881.0,16.333889,177.0,38876.0
1,38826.0,38865.0,38881.0,38876.0,16.417778,177.0,38911.0
2,38865.0,38881.0,38876.0,38911.0,16.500833,177.0,38829.0
3,38881.0,38876.0,38911.0,38829.0,16.583611,177.0,38735.0
4,38876.0,38911.0,38829.0,38735.0,16.667222,177.0,38518.0


In [5]:
df_wind = pd.DataFrame(Wind_train, 
             columns=['y(t-4)','y(t-3)','y(t-2)','y(t-1)','Hour','Day', 'y(t)'])
df_wind.to_csv('./Wind_train.csv',index=False)

df_wind = pd.DataFrame(Wind_val, 
             columns=['y(t-4)','y(t-3)','y(t-2)','y(t-1)','Hour','Day', 'y(t)'])
df_wind.to_csv('./Wind_val.csv',index=False)

df_wind = pd.DataFrame(Wind_test, 
             columns=['y(t-4)','y(t-3)','y(t-2)','y(t-1)','Hour','Day', 'y(t)'])
df_wind.to_csv('./Wind_test.csv',index=False)

df_wind.head()

,y(t-4),y(t-3),y(t-2),y(t-1),Hour,Day,y(t)
0,1322.0,1364.0,1370.0,1397.0,16.333889,177.0,1451.0
1,1364.0,1370.0,1397.0,1451.0,16.417778,177.0,1490.0
2,1370.0,1397.0,1451.0,1490.0,16.500833,177.0,1529.0
3,1397.0,1451.0,1490.0,1529.0,16.583611,177.0,1561.0
4,1451.0,1490.0,1529.0,1561.0,16.667222,177.0,1573.0
